In [44]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /root/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Package averaged_perceptron_tagger is already up-to-date!
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Package averaged_perceptron_tagger_ru is already up-to-
       |       date!
       | Downloading package basque_grammars to /root/nltk_data...
       |   Package basque_grammars is already up-to-date!


True

In [46]:
df=pd.read_csv("/content/drive/MyDrive/NLP/spam.csv",encoding="ISO-8859-1")
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [47]:
df=df.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"],axis=1)

In [48]:
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [49]:
lemmetizer=WordNetLemmatizer()
stop_words=stopwords.words("english")
def text_preprocessing(text):
  text = text.lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)
  text=word_tokenize(text)
  filtered_text = [w for w in text if not w in stop_words]
  lemmetized_text=[lemmetizer.lemmatize(w) for w in filtered_text]
  return lemmetized_text

In [38]:
df['v2']=df['v2'].apply(text_preprocessing)

In [39]:
df

,v1,v2
0,ham,"[go, jurong, point, ,, crazy.., available, bug..."
1,ham,"[ok, lar, ..., joking, wif, u, oni, ...]"
2,spam,"[free, entry, wkly, comp, win, fa, cup, final,..."
3,ham,"[u, dun, say, early, hor, ..., u, c, already, ..."
4,ham,"[nah, n't, think, go, usf, ,, life, around, th..."
...,...,...
5567,spam,"[time, tried, contact, u., u, å£, pound, prize..."
5568,ham,"[ì_, b, going, esplanade, fr, home, ?]"
5569,ham,"[pity, ,, *, mood, ., ..., suggestion, ?]"
5570,ham,"[guy, bitching, acted, like, 'd, interested, b..."


In [27]:
le=LabelEncoder()
df['v1']=le.fit_transform(df['v1'])

In [28]:
df.head()

,v1,v2
0,0,"go jurong point , crazy.. available bugis n gr..."
1,0,ok lar ... joking wif u oni ...
2,1,free entry wkly comp win fa cup final tkts may...
3,0,u dun say early hor ... u c already say ...
4,0,"nah n't think go usf , life around though"


In [50]:
Vectorizer=TfidfVectorizer()
X=Vectorizer.fit_transform(df['v2'])
X

<5572x8672 sparse matrix of type '<class 'numpy.float64'>'
	with 73916 stored elements in Compressed Sparse Row format>

In [52]:
Y=df['v1']
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=101)

In [57]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix,classification_report
nb = MultinomialNB()
nb.fit(X_train, Y_train)
nb_pred = nb.predict(X_test)

In [59]:
confusion_matrix(Y_test, nb_pred)

array([[1464,    0],
       [  62,  146]])

In [61]:
print(classification_report(Y_test, nb_pred))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1464
        spam       1.00      0.70      0.82       208

    accuracy                           0.96      1672
   macro avg       0.98      0.85      0.90      1672
weighted avg       0.96      0.96      0.96      1672



In [63]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train,Y_train)
pred2= rfc.predict(X_test)
print(confusion_matrix(Y_test,pred2))

[[1464    0]
 [  38  170]]


In [64]:
print(classification_report(Y_test, pred2))

              precision    recall  f1-score   support

         ham       0.97      1.00      0.99      1464
        spam       1.00      0.82      0.90       208

    accuracy                           0.98      1672
   macro avg       0.99      0.91      0.94      1672
weighted avg       0.98      0.98      0.98      1672

